In [1]:
import pandas as pd
import time
import numpy as np
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
columns = [
        'id','host_is_superhost', 'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates',
        'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities',
        'price', 'guests_included', 'extra_people', 'minimum_nights',
        'maximum_nights', 'number_of_reviews', 'cancellation_policy',
    ]

df = pd.read_csv('listings_details.csv', index_col='id', usecols=columns)
df = df.dropna()
drop_indices = np.random.choice(df.index, 2, replace=False)
df = df.drop(drop_indices)

In [7]:
df

superhost                           neighbourhood property_type  \
id                                                                          
2818              1  Oostelijk Havengebied - Indische Buurt     Apartment   
3209              0                              Westerpark     Apartment   
20168             0                            Centrum-Oost     Townhouse   
25428             0                            Centrum-West     Apartment   
27886             1                            Centrum-West     Houseboat   
...             ...                                     ...           ...   
30576148          0                         Watergraafsmeer         House   
30577727          0  Oostelijk Havengebied - Indische Buurt     Apartment   
30578037          0                                Oud-Oost     Apartment   
30579673          0  Oostelijk Havengebied - Indische Buurt     Apartment   
30580413          0                 De Pijp - Rivierenbuurt     Apartment   

                room_type  accommodates  bathrooms  bedrooms  beds  bed_type  \
id                                                                             
2818         Private room             2        1.5       1.0   2.0  Real Bed   
3209      Entire home/apt             5        1.0       2.0   2.0  Real Bed   
20168     Entire home/apt             2        1.0       1.0   1.0  Real Bed   
25428     Entire home/apt             3        1.0       1.0   1.0  Real Bed   
27886        Private room             2        1.0       1.0   1.0  Real Bed   
...                   ...           ...        ...       ...   ...       ...   
30576148  Entire home/apt             4        2.0       3.0   4.0  Real Bed   
30577727  Entire home/apt             3        1.0       2.0   3.0  Real Bed   
30578037  Entire home/apt             2        1.0       1.0   1.0  Real Bed   
30579673     Private room             2        1.0       1.0   1.0  Real Bed   
30580413  Entire home/apt             4        1.0       2.0   2.0  Real Bed   

                                                  amenities  price  \
id                                                                   
2818      {Internet,Wifi,"Paid parking off premises","Bu...   59.0   
3209      {Internet,Wifi,Kitchen,"Paid parking off premi...  160.0   
20168     {TV,Internet,Wifi,"Paid parking off premises",...   80.0   
25428     {TV,"Cable TV",Internet,Wifi,Kitchen,Elevator,...  125.0   
27886     {TV,Internet,Wifi,Breakfast,Heating,"Smoke det...  150.0   
...                                                     ...    ...   
30576148  {TV,Wifi,Kitchen,"Free parking on premises",He...  340.0   
30577727     {TV,Wifi,Kitchen,Essentials,"Hair dryer",Iron}  150.0   
30578037  {TV,"Cable TV",Wifi,Kitchen,"Smoking allowed",...   80.0   
30579673     {TV,Wifi,Kitchen,Essentials,"Hair dryer",Iron}   55.0   
30580413  {TV,Wifi,Kitchen,"Pets allowed",Heating,Washer...  220.0   

          guests_included  extra_people  minimum_nights  maximum_nights  \
id                                                                        
2818                    1          20.0               3              15   
3209                    2          15.0               4              20   
20168                   2           0.0               1            1000   
25428                   2          10.0              14              60   
27886                   1           0.0               2             730   
...                   ...           ...             ...             ...   
30576148                1           0.0               7              12   
30577727                1           0.0               3              14   
30578037                1           0.0              10              22   
30579673                1           0.0               2              15   
30580413                1           0.0               5            1125   

          number_of_reviews          cancellation_policy  
id                   

In [5]:
# rename columns
df = df.rename(columns={'host_is_superhost':'superhost', 'neighbourhood_cleansed':'neighbourhood'})

df['superhost'] = df.superhost.replace({'t':1, 'f':0})

In [6]:
df["price"] = pd.to_numeric(df["price"].str.strip("$").str.replace(",", ""), downcast="float")
df["extra_people"] = pd.to_numeric(df["extra_people"].str.strip("$").str.replace(",", ""), downcast="float")

In [150]:
# make sure amenities are split properly
def get_info(x):
    return x.strip("{|}").replace('"', '').split(",")

df['len_amenities'] = df['amenities'].apply(lambda x:len(get_info(x)))
df['amenities'] = df['amenities'].apply(lambda x:get_info(x))

In [151]:
mlb = MultiLabelBinarizer()
new_df = pd.DataFrame(mlb.fit_transform(df.pop('amenities')),
                          columns=mlb.classes_,
                          index=df.index).add_prefix('amenities_')

In [152]:
categorical_columns = ['neighbourhood', 'property_type', 'room_type', 'bed_type', 'cancellation_policy']

In [153]:
for col in categorical_columns:
    df[col] = df[col].astype('category')
df = pd.get_dummies(df)

In [155]:
final_df = df.join(new_df)

In [157]:
final_df.to_csv("cleaned_listings.csv")